<a href="https://colab.research.google.com/github/Rogerio-mack/BIG_DATA_Analytics_Mineracao_e_Analise_de_Dados/blob/main/BIG_A1_LinearRegressionScikitlearn_solucao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<head>
  <meta name="author" content="Rogério de Oliveira">
  <meta institution="author" content="Universidade Presbiteriana Mackenzie">
</head>

<img src="http://meusite.mackenzie.br/rogerio/mackenzie_logo/UPM.2_horizontal_vermelho.jpg" width=300, align="right">
<!-- <h1 align=left><font size = 6, style="color:rgb(200,0,0)"> optional title </font></h1> -->


# **Tarefa de Aprofundamento 1** 
## CASO: Estimando o Preço de Imóveis em São Paulo
---

Solução com o `scikit-learn`.


# Basic imports

In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Exploração dos Dados

Entenda os dados disponíveis e em seguida faça as transformações necessárias nos dados. Quantidades dos dados, se existem valores nulos e valores máximos e mínimos dos atributos são algumas explorações comuns. 

(Moodle) **Q1**. Quantos bairros existem no histórico e qual o bairro mais presente nos dados?



In [20]:
df = pd.read_excel('http://meusite.mackenzie.br/rogerio/data_load/regressao_preco_imoveis.xlsx')
df.head()

,bairro,areaM2,suites,dormitorios,banheiros,vagas,preco
0,vila-nova-conceicao,32,1,1,1,1,490000
1,vila-nova-conceicao,157,2,2,2,2,3180000
2,vila-nova-conceicao,205,2,3,3,3,1900000
3,vila-nova-conceicao,193,3,3,3,3,3565000
4,vila-nova-conceicao,116,1,3,2,2,1605000


In [21]:
# seu código

In [22]:
df.shape

(3741, 7)

In [23]:
df.isnull().sum()

bairro         0
areaM2         0
suites         0
dormitorios    0
banheiros      0
vagas          0
preco          0
dtype: int64

In [24]:
df.bairro.nunique()

5

In [25]:
df.bairro.value_counts()

vila-mariana           1714
paraiso                 802
vila-nova-conceicao     789
vila-sonia              307
morumbi                 129
Name: bairro, dtype: int64

In [26]:
df.describe(include='all')

,bairro,areaM2,suites,dormitorios,banheiros,vagas,preco
count,3741,3741.000000,3741.000000,3741.000000,3741.000000,3741.000000,3.741000e+03
unique,5,NaN,NaN,NaN,NaN,NaN,NaN
top,vila-mariana,NaN,NaN,NaN,NaN,NaN,NaN
freq,1714,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,140.343758,1.700882,2.946271,3.095429,2.225341,1.553746e+06
std,NaN,60.051791,0.994945,0.759292,1.372859,1.000940,1.073607e+06
min,NaN,20.000000,1.000000,1.000000,1.000000,1.000000,2.800000e+05
25%,NaN,94.000000,1.000000,3.000000,2.000000,1.000000,8.500000e+05
50%,NaN,130.000000,1.000000,3.000000,3.000000,2.000000,1.200000e+06
75%,NaN,180.000000,2.000000,3.000000,4.000000,3.000000,1.880000e+06


## Preparação 



### Hot encode



In [27]:
from sklearn.preprocessing import OneHotEncoder

hot_encode = OneHotEncoder(handle_unknown='ignore',drop='first') 
hot_encode = hot_encode.fit(df[['bairro']])

display(hot_encode.categories_)

hot = pd.DataFrame(hot_encode.transform(df[['bairro']] ).toarray(), columns = hot_encode.categories_[0][1::])  

df = pd.concat( [df.drop(columns=['bairro']), hot],axis=1)
df.head()

[array(['morumbi', 'paraiso', 'vila-mariana', 'vila-nova-conceicao',
        'vila-sonia'], dtype=object)]

,areaM2,suites,dormitorios,banheiros,vagas,preco,paraiso,vila-mariana,vila-nova-conceicao,vila-sonia
0,32,1,1,1,1,490000,0.0,0.0,1.0,0.0
1,157,2,2,2,2,3180000,0.0,0.0,1.0,0.0
2,205,2,3,3,3,1900000,0.0,0.0,1.0,0.0
3,193,3,3,3,3,3565000,0.0,0.0,1.0,0.0
4,116,1,3,2,2,1605000,0.0,0.0,1.0,0.0


## normalização 

In [28]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(df.drop(columns='preco')) 

X = scaler.transform(df.drop(columns='preco'))
y = df.preco

# Estimando o modelo com o `scikit-learn`

Estime o modelo.

(Moodle) **Q2**. Qual o coeficiente de determinação ajustado obtido? *não disponível diretamente no `scikit-learn`*

(Moodle) **Q3**. Qual coeficiente estimado é menos significativo? *não disponível diretamente no `scikit-learn`*

In [29]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X,y)

print('Coeficientes: ')
for i in range(len(model.coef_)):
  print(list(df.drop(columns='preco').columns)[i], ': \t' , model.coef_[i])

print('\nIntercept: ')
print(model.intercept_)

print()
print(f'Coeficiente de Determinação {model.score(X,y): .5f}')


Coeficientes: 
areaM2 : 	 455149.37062877626
suites : 	 152426.55283114203
dormitorios : 	 -75697.6251685996
banheiros : 	 6528.433150484998
vagas : 	 333345.51039376826
paraiso : 	 34189.10436118475
vila-mariana : 	 -58802.620837612485
vila-nova-conceicao : 	 314037.36288538226
vila-sonia : 	 -143958.77176795367

Intercept: 
1553745.5875434375

Coeficiente de Determinação  0.79531


# Re-estimando o modelo com o `statsmodels`

Estime novamente o modelo excluindo o atributo menos significativo (um único atributo). 

(Moodle) **Q4**. Qual o novo coeficiente de determinação ajustado obtido? *não disponível diretamente no `scikit-learn`*


In [31]:
from sklearn.linear_model import LinearRegression

scaler = StandardScaler()
scaler.fit(df.drop(columns=['preco','banheiros'])) 

X = scaler.transform(df.drop(columns=['preco','banheiros']))
y = df.preco

model = LinearRegression()
model.fit(X,y)

print('Coeficientes: ')
for i in range(len(model.coef_)):
  print(list(df.drop(columns='preco').columns)[i], ': \t' , model.coef_[i])

print('\nIntercept: ')
print(model.intercept_)

print()
print(f'Coeficiente de Determinação {model.score(X,y): .5f}')

Coeficientes: 
areaM2 : 	 455851.59684183245
suites : 	 155289.8051678064
dormitorios : 	 -74361.72996696527
banheiros : 	 333970.1348948227
vagas : 	 34010.66622162518
paraiso : 	 -58848.49169483243
vila-mariana : 	 313690.6880935808
vila-nova-conceicao : 	 -143989.47773898856

Intercept: 
1553745.5875434375

Coeficiente de Determinação  0.79529


# Predição

Faça a predição do preço, com o melhor dos modelos obtidos acima, de um imóvel na Vila Mariana, com 134 m2, 1 suite, 4 dorm, 3 banheiros e 3 vagas.



(Moodle) **Q5**. Qual o preço estimado, para o melhor dos modelos obtidos, de um imóvel na Vila Mariana, com 134 m2, 1 suite, 4 dorm, 3 banheiros e 3 vagas?



Dica: Lembre que as *mesmas transformações* de hot encode e de normalização precisam ser aplicadas aos dados.

In [33]:
df_case = pd.DataFrame(data=None, index=[0])

df_case['areaM2'] = 134
df_case['suites'] = 1
df_case['dormitorios'] = 4
# df_case['banheiros'] = 3
df_case['vagas'] = 3 
df_case['paraiso'] = 0
df_case['vila-mariana'] = 1
df_case['vila-nova-conceicao'] = 0
df_case['vila-sonia'] = 0

df_case

,areaM2,suites,dormitorios,vagas,paraiso,vila-mariana,vila-nova-conceicao,vila-sonia
0,134,1,4,3,0,1,0,0


In [34]:
X_test = scaler.transform(df_case)
preco = model.predict(X_test)

print(f'Preco estimado: {preco[0]:.2f}')

Preco estimado: 1350584.80
